# PreSeizure Project — CHB-MIT Loader

This Colab-ready notebook downloads a small subset of the CHB-MIT EEG dataset from PhysioNet and loads an EDF file with `wfdb`. Upload this notebook to Google Colab (`File → Upload notebook`) or open it in Colab directly after downloading.

⚠️ Note: the full CHB-MIT dataset is large (~4 GB). This notebook downloads only a few example files (safe for Colab). If you want the full dataset, mount Google Drive and store files there.

In [ ]:
# =====================================
# PreSeizure Project - CHB-MIT Loader
# =====================================

# Step 1. Install required libraries
!pip install --quiet wfdb neurokit2

# Step 2. Imports
import os
import requests
import wfdb

# Step 3. Create dataset folder
os.makedirs("CHBMIT_dataset", exist_ok=True)

# Step 4. Define PhysioNet base URL
base_url = "https://physionet.org/content/chbmit/1.0.0/"

# Step 5. Download a few example files (safe for Colab)
patients = ["chb01", "chb02"]
sessions = ["01", "02", "03"]

downloaded_files = []

for patient in patients:
    for sess in sessions:
        file_rel = f"{patient}/{patient}_{sess}.edf"
        file_url = base_url + file_rel
        local_file = os.path.join("CHBMIT_dataset", os.path.basename(file_rel))

        if not os.path.exists(local_file):
            try:
                print("Downloading:", file_url)
                r = requests.get(file_url, stream=True)
                r.raise_for_status()
                with open(local_file, "wb") as f:
                    for chunk in r.iter_content(1024 * 1024):
                        f.write(chunk)
                print("✅ Saved to", local_file)
            except Exception as e:
                print("⚠️ Skipped:", file_url, "| Error:", e)
                continue
        else:
            print("Already exists:", local_file)

        downloaded_files.append(local_file)

print("\nDownloaded files:", downloaded_files)

# Step 6. Load one file with wfdb
if downloaded_files:
    record = wfdb.rdrecord(downloaded_files[0])
    print("\nChannels:", record.sig_name)
    print("Signal shape:", record.p_signal.shape)
else:
    print("No files downloaded. Check connection or file paths.")
